In [1]:
from huggingface_hub import notebook_login
notebook_login(new_session=False)

User is already logged in.


# DATA LOAD

In [2]:
# from transformers import AutoTokenizer
# import json 
# import os
# from tqdm.auto import tqdm 
# import datasets
# wiki_path = '../input/data/wikipedia_documents.json'
# with open(wiki_path, 'r',encoding='utf-8') as f:
#     wiki = json.load(f)

# PREPROCESSING

In [ ]:
# context = list(dict.fromkeys([v['text'] for v in wiki.values()]))
# import re 
# from unidecode import unidecode
# from datasets import Dataset
# context = [{'text':re.sub(r'([^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z\d\s\.\,\'\"\<\>\!\@\#\$\%\^\&\*\(\)\_\+\-])',lambda x:unidecode(x.group()),c)} for c in context]
# context.pop(37157) # 가각갂간ㅅ .....
# raw_dataset = Dataset.from_list(context)
# tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

# TOKENIZER

In [3]:
# tokenizer_id = 'dnjdsxor21/wiki-tokenizer'

# def batch_iterator(batch_size=10000):
#     for i in tqdm(range(0, len(context), batch_size)):
#         yield raw_dataset[i : i + batch_size]['text']
# # roberta_tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
# # roberta_tokenizer = tokenizer.train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=32000)
# # roberta_tokenizer.save_pretrained('custom_tokenizer')
# roberta_tokenizer = AutoTokenizer.from_pretrained('./custom_tokenizer/')
# roberta_tokenizer

BertTokenizerFast(name_or_path='./custom_tokenizer/', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [4]:
# roberta_tokenizer.push_to_hub(tokenizer_id)

CommitInfo(commit_url='https://huggingface.co/dnjdsxor21/wiki-tokenizer/commit/e2009debe0a5e6f32d852ee95c9457cbea54a796', commit_message='Upload tokenizer', commit_description='', oid='e2009debe0a5e6f32d852ee95c9457cbea54a796', pr_url=None, pr_revision=None, pr_num=None)

# DATA

In [ ]:
# import multiprocessing

# tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
# num_proc = int(multiprocessing.cpu_count() / 2)
# print('max length for tokenizer', tokenizer.model_max_length)

# def group_texts(examples):
#     tokenized_inputs = tokenizer(
#         examples['text'], truncation=True, max_length=tokenizer.model_max_length, return_special_tokens_mask=True, stride=128
#     )
#     return tokenized_inputs
# tokenized_dataset = raw_dataset.map(group_texts, batched=True, remove_columns=['text'], num_proc=num_proc)
# print(len(raw_dataset), len(tokenized_dataset))


In [ ]:
# dataset_id = 'dnjdsxor21/preprocessed-wiki-kor'
# tokenized_dataset.push_to_hub(dataset_id)

# Pretrain 시작

In [1]:
# 빈 디렉토리 제거
import os
dirs = os.listdir('./models')
for dir in dirs:
    path = f'./models/{dir}'
    if not os.listdir(path):
        os.rmdir(path)

In [1]:
from transformers import (Trainer, TrainingArguments, RobertaForMaskedLM, AutoTokenizer)
from data_collator import DataCollatorForLanguageModeling
from datasets import load_dataset,Dataset
from sklearn.metrics import f1_score
import numpy as np
import random
import torch
import wandb
from datetime import datetime, timezone, timedelta
import os 

torch.cuda.empty_cache()

tokenizer_id ='dnjdsxor21/roberta-klue-ssm'
dataset_id = 'dnjdsxor21/preprocessed-wiki-kor'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
dataset = load_dataset(dataset_id)


WANDB_NAME = datetime.now(tz=timezone(timedelta(hours=9))).strftime("%m-%d-%H:%M:%S")
wandb.init(project='Pretraining',
               name= WANDB_NAME,
               )
OUTPUT_DIR = f'./models/{WANDB_NAME}'
if not os.path.isdir(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

training_args = TrainingArguments(
    output_dir = OUTPUT_DIR,
    overwrite_output_dir=True,
    do_train=True,
    logging_strategy='steps',
    num_train_epochs=100,
    logging_steps=100,
    per_device_train_batch_size=32,
    learning_rate=5e-5,
    dataloader_num_workers=4,
    save_total_limit=2,
    seed=42
)
model = RobertaForMaskedLM.from_pretrained('klue/roberta-base')

data_collator = DataCollatorForLanguageModeling(  # span masking
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15, pad_to_multiple_of=8
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    f1 = f1_score(y_true=labels, y_pred=predictions, average='micro')
    return {'f1':f1}

trainer = Trainer(
    model = model, 
    args=training_args,
    train_dataset=dataset['train'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Found cached dataset parquet (/opt/ml/.cache/huggingface/datasets/dnjdsxor21___parquet/dnjdsxor21--preprocessed-wiki-kor-d65d7186795135d1/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dnjdsxor21 (bcnlp111). Use `wandb login --relogin` to force relogin


In [6]:
checkpoint = './models/06-12-05:51:37/checkpoint-33500'
trainer.train(resume_from_checkpoint=checkpoint if os.listdir(checkpoint) else None)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You'

Step,Training Loss
33600,1.295200
33700,1.317700
33800,1.311900
33900,1.305100
34000,1.316000
34100,1.286400
34200,1.319100
34300,1.300400
34400,1.326800
34500,1.327200


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `_

KeyboardInterrupt: 

In [ ]:
# model save (local)
trainer.save_model(OUTPUT_DIR) 

In [2]:
# model save (repo)
model = RobertaForMaskedLM.from_pretrained('./models/06-12-17:35:49/checkpoint-47000')
model.push_to_hub('dnjdsxor21/roberta-klue-ssm')

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dnjdsxor21/roberta-klue-ssm/commit/c7489c64d4a4eccaa1c7978f2a2736f7865f2341', commit_message='Upload RobertaForMaskedLM', commit_description='', oid='c7489c64d4a4eccaa1c7978f2a2736f7865f2341', pr_url=None, pr_revision=None, pr_num=None)